Loading dataset

In [14]:
import pandas as pd
path = 'C:/Users/maksd/OneDrive/Pulpit/ASI/asi-project/data/'

df = pd.read_csv(path + 'Extended_Employee_Performance_and_Productivity_Data.csv')
print(df.shape)
df.head()

(100000, 20)


,Employee_ID,Department,Gender,Age,Job_Title,Hire_Date,Years_At_Company,Education_Level,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Overtime_Hours,Sick_Days,Remote_Work_Frequency,Team_Size,Training_Hours,Promotions,Employee_Satisfaction_Score,Resigned
0,1,IT,Male,55,Specialist,2022-01-19 08:03:05.556036,2,High School,5,6750.0,33,32,22,2,0,14,66,0,2.63,False
1,2,Finance,Male,29,Developer,2024-04-18 08:03:05.556036,0,High School,5,7500.0,34,34,13,14,100,12,61,2,1.72,False
2,3,Finance,Male,55,Specialist,2015-10-26 08:03:05.556036,8,High School,3,5850.0,37,27,6,3,50,10,1,0,3.17,False
3,4,Customer Support,Female,48,Analyst,2016-10-22 08:03:05.556036,7,Bachelor,2,4800.0,52,10,28,12,100,10,0,1,1.86,False
4,5,Engineering,Female,36,Analyst,2021-07-23 08:03:05.556036,3,Bachelor,2,4800.0,38,11,29,13,100,15,9,1,1.25,False


Data exploartion

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Employee_ID                  100000 non-null  int64  
 1   Department                   100000 non-null  object 
 2   Gender                       100000 non-null  object 
 3   Age                          100000 non-null  int64  
 4   Job_Title                    100000 non-null  object 
 5   Hire_Date                    100000 non-null  object 
 6   Years_At_Company             100000 non-null  int64  
 7   Education_Level              100000 non-null  object 
 8   Performance_Score            100000 non-null  int64  
 9   Monthly_Salary               100000 non-null  float64
 10  Work_Hours_Per_Week          100000 non-null  int64  
 11  Projects_Handled             100000 non-null  int64  
 12  Overtime_Hours               100000 non-null  int64  
 13  

In [16]:
df.isnull().sum().sort_values(ascending=False)/df.shape[0]

Employee_ID                    0.0
Department                     0.0
Employee_Satisfaction_Score    0.0
Promotions                     0.0
Training_Hours                 0.0
Team_Size                      0.0
Remote_Work_Frequency          0.0
Sick_Days                      0.0
Overtime_Hours                 0.0
Projects_Handled               0.0
Work_Hours_Per_Week            0.0
Monthly_Salary                 0.0
Performance_Score              0.0
Education_Level                0.0
Years_At_Company               0.0
Hire_Date                      0.0
Job_Title                      0.0
Age                            0.0
Gender                         0.0
Resigned                       0.0
dtype: float64

Data cleaning/Encoding Categorical Variables/Scaling values in dataset

In [17]:
df = df.drop(['Employee_ID'], axis=1)
df = df.drop(['Hire_Date'], axis=1)

In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['Department'] = label_encoder.fit_transform(df['Department'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Job_Title'] = label_encoder.fit_transform(df['Job_Title'])
#df['Hire_Date'] = pd.to_datetime(df['Hire_Date'])
df['Education_Level'] = label_encoder.fit_transform(df['Education_Level'])
#df['Hire_Date'] = df['Hire_Date'].dt.year

In [19]:
from sklearn.preprocessing import MinMaxScaler
df['Overtime_Ratio'] = ((df['Overtime_Hours'] + df['Work_Hours_Per_Week']) / df['Work_Hours_Per_Week'])
scaler = MinMaxScaler()
df['Overtime_Ratio'] = scaler.fit_transform(df[['Overtime_Ratio']])

Feature and Target Variable Selection

In [20]:
y = df['Employee_Satisfaction_Score']
x = df.drop('Employee_Satisfaction_Score', axis=1)

In [21]:
y.head()

0    2.63
1    1.72
2    3.17
3    1.86
4    1.25
Name: Employee_Satisfaction_Score, dtype: float64

In [22]:
x.head()

,Department,Gender,Age,Job_Title,Years_At_Company,Education_Level,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Overtime_Hours,Sick_Days,Remote_Work_Frequency,Team_Size,Training_Hours,Promotions,Resigned,Overtime_Ratio
0,4,1,55,5,2,1,5,6750.0,33,32,22,2,0,14,66,0,False,0.689655
1,2,1,29,2,0,1,5,7500.0,34,34,13,14,100,12,61,2,False,0.395538
2,2,1,55,5,8,1,3,5850.0,37,27,6,3,50,10,1,0,False,0.167754
3,0,0,48,0,7,0,2,4800.0,52,10,28,12,100,10,0,1,False,0.557029
4,1,0,36,0,3,0,2,4800.0,38,11,29,13,100,15,9,1,False,0.789474


Divide the dataset into training and test set

In [23]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)

Linear Regression Model training and evaluation

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

y_pred = lin_reg.predict(x_test)
print(y_pred[4])
print(y_test.iloc[4])
rmse = mean_squared_error(y_test, y_pred, squared = False)
r2_score = lin_reg.score(x_test, y_test)
print ('Root Means Square Error = ', rmse)
print(f"R² Score: {r2_score}")

2.985972571134318
3.68
Root Means Square Error =  1.149984025071542
R² Score: -0.00043495322126863023


Decision Tree Regressor Model training and evaluation 

In [25]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(x_train, y_train)

y_pred = regressor.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
r2_score = regressor.score(x_test, y_test)
print(f"Root Mean Squared Error: {rmse}")
print(f"R² Score: {r2_score}")

Root Mean Squared Error: 1.647604401345582
R² Score: -1.0535776485271064
